In [1]:
trainFolderPath = "C:/Users/kpraj/Desktop/Multi-label Image Classification on Medical Scans/dataset/chest_xray/SPLIT/train"
testFolderPath = "C:/Users/kpraj/Desktop/Multi-label Image Classification on Medical Scans/dataset/chest_xray/SPLIT/test"
validationFolderPath = "C:/Users/kpraj/Desktop/Multi-label Image Classification on Medical Scans/dataset/chest_xray/SPLIT/val"

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

<h2>Pre Processing Block</h2>

Objective:<br><ul>
<li>Resize all images to 32x32 pixels</li>
<li>Normalize image pixel values (e.g., scale between 0 and 1 or standardize)</li>
<li>Return image data as tensors</li>
<li>Return corresponding image classes (labels)</li>

In [13]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def preProcessingImage(path):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),
                             (0.5, 0.5, 0.5))
    ])

    dataset = datasets.ImageFolder(root=path, transform=transform)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

    return dataloader, dataset.classes

<h2>Model Block</h2>

<ul>
  <li>We are using a Convolutional Neural Network (CNN) for this task. The model accepts 32x32 images, as processed by the pre-processing block.</li>
  <li>The network has 2 convolutional layers with a kernel size of 5x5.</li>
  <li>Max pooling is performed with a 2x2 kernel.</li>
  <li>The architecture transitions from convolutional layers to fully connected layers for classification.</li>
</ul>


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self._to_linear = None
        self._get_conv_output()

        self.fc1 = nn.Linear(self._to_linear, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def _get_conv_output(self):
        x = torch.randn(1, 3, 32, 32)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        self._to_linear = x.numel()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

<p>We use the <code>CrossEntropyLoss</code> as the loss function for classification tasks.</p>
<p>The optimizer chosen is Stochastic Gradient Descent (SGD) with a learning rate of 0.001 and momentum of 0.9, applied to the network's parameters.</p>


In [15]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

<h2>Training</h2>

In [16]:
trainloader, trainClasses = preProcessingImage(trainFolderPath)
testloader, testClasses = preProcessingImage(testFolderPath)
validationloader, valClasses = preProcessingImage(validationFolderPath)

print("Training Classes:" ,trainClasses)
print("Testing Classes:" ,testClasses)
print("Validation Classes:" ,valClasses)


for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Training Classes: ['NORMAL', 'PNEUMONIA']
Testing Classes: ['NORMAL', 'PNEUMONIA']
Validation Classes: ['NORMAL', 'PNEUMONIA']
[1,  2000] loss: 0.577
[1,  4000] loss: 0.491
[2,  2000] loss: 0.471
[2,  4000] loss: 0.448
Finished Training


In [17]:
torch.save(net.state_dict(), 'model.pth')
print("Model saved as model.pth")

Model saved as model.pth


<h2>Testing</h2>

In [18]:
net = Net()
net.load_state_dict(torch.load('model.pth'))
net.eval()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in testloader:
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


Accuracy on test set: 78.91%


<h2>Validation</h2>

In [19]:
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in validationloader:
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct / total:.2f}%')


Validation Accuracy: 78.36%


<h2>Final conclusion</h2>

Testing Accuracy: 78.91%<br>
Validation Accuracy: 78.36%

This is basic image classification for identifying the patient has pneumonia or he is normal based o his chest scans. Next level is with identifying the patient age, patient sex and patent condition based on image processing. 